Using boto3 to do blue/gree deployment

In [15]:
import boto3
from datetime import datetime

session = boto3.Session()
sm = session.client('sagemaker')

In [4]:
dev_endpoint_name = "direct-marketing-endpoint-dev"
prd_endpoint_name = "direct-marketing-endpoint-prd"

In [7]:
def get_endpoint_config(endpoint_name):
    response = sm.describe_endpoint(EndpointName = endpoint_name)
    endpoint_config_name = response['EndpointConfigName']
    return sm.describe_endpoint_config(EndpointConfigName = endpoint_config_name)


In [10]:
dev_endpoint_config = get_endpoint_config(dev_endpoint_name)
prd_endpoint_config = get_endpoint_config(prd_endpoint_name)

In [11]:
dev_model_name = dev_endpoint_config['ProductionVariants'][0]['ModelName']
dev_model_name

'dm-model-201129-031717'

In [17]:
prd_endpoint_config

{'EndpointConfigName': 'prd-endpoint-config',
 'EndpointConfigArn': 'arn:aws:sagemaker:ap-southeast-2:593380422482:endpoint-config/prd-endpoint-config',
 'ProductionVariants': [{'VariantName': 'default-variant-name',
   'ModelName': 'dm-model-201129-025107',
   'InitialInstanceCount': 1,
   'InstanceType': 'ml.m4.xlarge',
   'InitialVariantWeight': 1.0}],
 'CreationTime': datetime.datetime(2020, 11, 29, 4, 13, 50, 19000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '0acdd8a0-6a54-4404-b903-cd3637e82307',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0acdd8a0-6a54-4404-b903-cd3637e82307',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '361',
   'date': 'Sun, 29 Nov 2020 04:16:29 GMT'},
  'RetryAttempts': 0}}

In [20]:
prd_data_capture_config = prd_endpoint_config['DataCaptureConfig'] if 'DataCaptureConfig' in prd_endpoint_config else {}
prd_model_name = prd_endpoint_config['ProductionVariants'][0]['ModelName']
prd_model_name

'dm-model-201129-025107'

In [13]:
dev_prd_in_sync = (dev_model_name == prd_model_name)

In [14]:
dev_prd_in_sync

False

In [23]:
suffix = datetime.now().strftime("%y%m%d-%H%M%S")
endpoint_config_name = f"directmarketing-endpoint-config-{suffix}"

In [24]:
sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants = [
        {
            'VariantName': 'blue-variant',
            'ModelName': prd_model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
            'InitialVariantWeight': 9
        },    
        {
            'VariantName': 'green-variant',
            'ModelName': dev_model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
            'InitialVariantWeight': 1
       }
    ]
)

{'EndpointConfigArn': 'arn:aws:sagemaker:ap-southeast-2:593380422482:endpoint-config/directmarketing-endpoint-config-201129-043437',
 'ResponseMetadata': {'RequestId': '45d5dd76-052f-4e50-8309-6713c16d67ff',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '45d5dd76-052f-4e50-8309-6713c16d67ff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '131',
   'date': 'Sun, 29 Nov 2020 04:34:38 GMT'},
  'RetryAttempts': 0}}

In [25]:
response = sm.update_endpoint(
    EndpointName = prd_endpoint_name,
    EndpointConfigName = endpoint_config_name,
    RetainAllVariantProperties = False
)

In [27]:
response = sm.update_endpoint_weights_and_capacities(
    EndpointName = prd_endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'blue-variant',
            'DesiredWeight': 1
        },
        {
            'VariantName': 'green-variant',
            'DesiredWeight': 9
        }
    ]
)

In [28]:
response

{'EndpointArn': 'arn:aws:sagemaker:ap-southeast-2:593380422482:endpoint/direct-marketing-endpoint-prd',
 'ResponseMetadata': {'RequestId': '256101ee-a8b0-4617-9133-a3a5ae9a20b1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '256101ee-a8b0-4617-9133-a3a5ae9a20b1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Sun, 29 Nov 2020 05:00:29 GMT'},
  'RetryAttempts': 0}}

workflow for blue green deployment

In [30]:
from pipeline.ml_pipeline_dependencies import *

model_name = "direct-marketing-model-training"
# create bucket for model training
bucket_name = f'{model_name}-{region}-{account_id}'

In [66]:
# lambda function
import zipfile
from sagemaker.s3 import S3Uploader
zip_name = 'create_blue_gree_deployment_endpoint_config.zip'
lambda_source_code = './code/create_blue_gree_deployment_endpoint_config.py'

zf = zipfile.ZipFile(zip_name, mode='w')
zf.write(lambda_source_code, arcname=lambda_source_code.split('/')[-1])
zf.close()
S3Uploader.upload(
    local_path = zip_name, 
    desired_s3_uri = f"s3://{bucket_name}/code",
    sagemaker_session = sagemaker_session
)

's3://direct-marketing-model-training-ap-southeast-2-593380422482/code/create_blue_gree_deployment_endpoint_config.zip'

In [67]:
lambda_client = boto3.client('lambda')

lambda_function_create_blue_gree_deployment_endpoint_config = 'create_blue_gree_deployment_endpoint_config'
response = lambda_client.create_function(
    FunctionName = lambda_function_create_blue_gree_deployment_endpoint_config,
    Runtime = 'python3.7',
    Role = sagemaker_execution_role,
    Handler = 'create_blue_gree_deployment_endpoint_config.lambda_handler',
    Code={
        'S3Bucket': bucket_name,
        'S3Key': f'code/{zip_name}'
    },
    Description='Create Blue/Green Deployment Endpoint Config.',
    Timeout=15,
    MemorySize=128
)


In [68]:
def create_lambda_create_blue_gree_deployment_endpoint_config_step():
    lambda_step = LambdaStep(
        'Create Blue/Green Deployment Endpoint Config',
        parameters = {  
            "FunctionName.$": "$$.Execution.Input['LambdaFunctionNameOfCreateBlueGreenDeploymentEndpointConfig']",
            'Payload':{
                "EndpointConfigName.$": "$$.Execution.Input['EndpointConfigName']",
                "DevEndpointName.$": "$$.Execution.Input['DevEndpointName']",
                "PrdEndpointName.$": "$$.Execution.Input['PrdEndpointName']"
            }
        }
    )   
    return lambda_step

In [69]:
def create_endpoint_step(endpoint_name_placeholder, endpoint_config_name_placeholder, update = False):
    endpoint_step = EndpointStep(
        "Update Endpoint" if update else "Create Endpoint",
        endpoint_name = endpoint_name_placeholder,
        endpoint_config_name = endpoint_config_name_placeholder,
        update = update
    )
    return endpoint_step

In [73]:
def create_query_endpoint_deployment_lambda_step():
    query_endpoint_deployment_lambda_step = LambdaStep(
        'Query Endpoint Deployment Status',
        parameters = {  
            "FunctionName.$": "$$.Execution.Input['LambdaFunctionNameOfQueryEndpoint']",
            'Payload':{
                "EndpointName.$": "$$.Execution.Input['PrdEndpointName']"
            }
        }
    )
    return query_endpoint_deployment_lambda_step

In [38]:
def create_check_endpoint_is_deploying_choice_step(
    query_endpoint_deployment_lambda_step,
    success_notification_step
):
    # check endpoint readiness
    deployed_endpoint_updating_step = Choice('Endpoint is deploying?')

    wait_deployment_step = Wait(state_id = "Wait Until Deployment is Completed...", seconds = 20)
    wait_deployment_step.next(query_endpoint_deployment_lambda_step)

    deployed_endpoint_updating_rule = ChoiceRule.StringEquals(variable = query_endpoint_deployment_lambda_step.output()['Payload']['endpoint_status'], value = 'InService')
    deployed_endpoint_updating_step.add_choice(rule = deployed_endpoint_updating_rule, next_step = success_notification_step)
    
    deployed_endpoint_updating_step.default_choice(next_step = wait_deployment_step)

    return deployed_endpoint_updating_step


In [39]:
def create_success_notification_step(topic_arn, subject):
    success_sns_step = SnsPublishStep(
        state_id = 'SNS Notification - Pipeline Succeeded',
        parameters = {
            'TopicArn': topic_arn,
            'Message.$': "$$.Execution.Id",
            'Subject': subject
        }
    )    
    return success_sns_step 

In [42]:
def create_failure_notification_step(
    topic_arn
):
    failure_sns_step = SnsPublishStep(
        state_id = 'SNS Notification - Pipeline Failure',
        parameters = {
            'TopicArn': topic_arn,
            'Message.$': "$",
            'Subject': '[ML Pipeline] Execution failed...'
        }
    )    
    return failure_sns_step

In [46]:
def get_state_machine_arn(workflow_name, region, account_id):
    return f"arn:aws:states:{region}:{account_id}:stateMachine:{workflow_name}"

def is_workflow_existed(workflow_role_arn):
    try:
        sfn_client = boto3.client('stepfunctions')
        response = sfn_client.describe_state_machine(
            stateMachineArn = workflow_role_arn
        )
        return True
    except: 
        return False

In [49]:
def create_workflow(
    topic_name,
    workflow_name,
    region, 
    account_id,
    workflow_execution_role
):
    # Workflow Execution parameters
    execution_input = ExecutionInput(
        schema = {
            "EndpointConfigName": str,
            "DevEndpointName": str,
            "PrdEndpointName": str,
            "LambdaFunctionNameOfQueryEndpoint": str,
            "LambdaFunctionNameOfCreateBlueGreenDeploymentEndpointConfig": str
        }
    )

    # create the steps
    blue_green_endpoint_config_step = create_lambda_create_blue_gree_deployment_endpoint_config_step()
    endpoint_update_step = create_endpoint_step(execution_input["PrdEndpointName"], execution_input["EndpointConfigName"], True)
    
    # create the choice steps
    query_endpoint_deployment_lambda_step = create_query_endpoint_deployment_lambda_step()
    topic_arn = f"arn:aws:sns:{region}:{account_id}:{topic_name}"
    success_notification_step = create_success_notification_step(topic_arn, "[ML Pipeline] Blue/Green Deployment Endpoing is ready.")
    check_endpoint_is_deploying_choice_step = create_check_endpoint_is_deploying_choice_step(
        query_endpoint_deployment_lambda_step,
        success_notification_step
    )

    deployment_path = Chain(
        [
            blue_green_endpoint_config_step, 
            endpoint_update_step, 
            query_endpoint_deployment_lambda_step, 
            check_endpoint_is_deploying_choice_step
        ]
    )

    # catch execution exception
    failed_state_sagemaker_pipeline_step_failure = Fail(
        "ML Workflow Failed", cause = "SageMakerPipelineStepFailed"
    )
    failure_notification_step = create_failure_notification_step(topic_arn)
    
    catch_state_processing = Catch(
        error_equals = ["States.TaskFailed"],
        next_step = Chain([failure_notification_step, failed_state_sagemaker_pipeline_step_failure])
    )
    blue_green_endpoint_config_step.add_catch(catch_state_processing)
    endpoint_update_step.add_catch(catch_state_processing)
    query_endpoint_deployment_lambda_step.add_catch(catch_state_processing)
    
    # Create Workflow
    workflow_arn = get_state_machine_arn(workflow_name, region, account_id)
    workflow_existed = is_workflow_existed(workflow_arn)
    if workflow_existed:
        # To update SFN workflow, need to do 'attach' & 'update' together.
        workflow = Workflow.attach(state_machine_arn = workflow_arn)
        workflow.update(definition = deployment_path, role = workflow_execution_role) 
        # Wait for 10s so that the update is completed before executing workflow
        time.sleep(10)
    else:
        workflow = Workflow(
            name = workflow_name,
            definition = deployment_path,
            role = workflow_execution_role
        )
        workflow.create()
    return workflow

In [74]:
workflow_name = "directmarketing-prd-deployment"
topic_name = 'dm-mt-notification-topic'

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name = "/ml_pipeline/workflow_execution_role")
workflow_execution_role = response['Parameter']['Value']

# bucket_name is created in ml_pipeline_dependencies.py, which is imported at the beginning.
workflow = create_workflow(
    topic_name,
    workflow_name,
    region, 
    account_id,
    workflow_execution_role
)

INFO:stepfunctions:Workflow updated successfully on AWS Step Functions. All execute() calls will use the updated definition and role within a few seconds. 


workflow for final green deployment

In [72]:
# execute workflow
suffix = datetime.now().strftime("%y%m%d-%H%M")
endpoint_config_name = f"directmarketing-endpoint-config-{suffix}"
dev_endpoint_name = "direct-marketing-endpoint-dev"
prd_endpoint_name = "direct-marketing-endpoint-prd"

execution = workflow.execute(
    inputs = {
        "EndpointConfigName": endpoint_config_name,
        "DevEndpointName": dev_endpoint_name,
        "PrdEndpointName": prd_endpoint_name,
        "LambdaFunctionNameOfQueryEndpoint": "query_endpoint",
        "LambdaFunctionNameOfCreateBlueGreenDeploymentEndpointConfig": "create_blue_gree_deployment_endpoint_config"
    }
)

INFO:stepfunctions:Workflow execution started successfully on AWS Step Functions.
